In [ ]:
# kaggle initialization
!pip install kaggle
!mkdir .kaggle
import json

token = {"username":"zhuningbo","key":"bfc23d452a2bb1fd791e4237234827d4"}

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
# download data
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c modified-mnist -p /content
!ls -la
!unzip \*.zip

In [ ]:
# load data
import numpy as np
import pandas as pd
train_img = pd.read_pickle('train_max_x')
train_lbl = pd.read_csv('train_max_y.csv')
test_img = pd.read_pickle('test_max_x')

In [ ]:
# image display (for testing)
import matplotlib.pyplot as plt
index = 100
plt.imshow(test_img[index].reshape((128,128)))

In [ ]:
# Dataset split
from sklearn.model_selection import train_test_split

lbl = [train_lbl.iloc[ID]['Label'] for ID in range(len(train_lbl))]

train_x, testvalid_x, train_y, testvalid_y = train_test_split(train_img, lbl, test_size=0.1, shuffle=False)
valid_x, test_x, valid_y, test_y = train_test_split(testvalid_x, testvalid_y, test_size=0.1, shuffle=False)

train_x = np.array(train_x)
test_x = np.array(test_x)
valid_x = np.array(valid_x)


In [ ]:
# Preprocessing

!sudo apt-get install libhdf5-serial-dev
!mkdir ./output

import keras
from keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.constraints import maxnorm

import h5py 
from __future__ import print_function

num_classes = 10
batch_size = 100
epochs = 60

row, col = 128, 128

x_train = np.concatenate( [train_x] ).astype('float32').reshape(-1,row,col,1)
x_valid = valid_x.astype('float32').reshape(-1,row,col,1)
x_test = test_img.astype('float32').reshape(-1,row,col,1)
rgb_x_valid = np.repeat(valid_x[..., np.newaxis], 3, -1)
rgb_x_test = np.repeat(test_img[..., np.newaxis], 3, -1)
print(x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'test samples')

y_train = keras.utils.to_categorical(np.concatenate([train_y]), num_classes)
y_valid = keras.utils.to_categorical(valid_y, num_classes)

input_shape = (row, col, 1)

datagen = ImageDataGenerator(rotation_range=20,
                shear_range=0.1,
                zoom_range=0.1,
                horizontal_flip=False,
                fill_mode='nearest')
datagen.fit(x_train)
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)

In [ ]:
# Model definition
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
import cv2

model = Sequential()
dr_c = 0
dr_n = 0.1

# model.add(MaxPooling2D((2,2), strides=(2,2), input_shape=(128,128,1)))

model.add(ZeroPadding2D((1,1),input_shape=(128,128,1)))
# model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(Dropout(dr_c))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128,(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(Dropout(dr_c))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(Dropout(dr_c))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(Dropout(dr_c))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(Dropout(dr_c))

model.add(Flatten())
model.add(Dense(64*64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(dr_n))

model.add(Dense(64*64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(dr_n))
model.add(Dense(10, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
      optimizer=keras.optimizers.Adam(lr=0.001,epsilon=1e-3),
      metrics=['accuracy'])

In [ ]:
# load weights from a previously trained model
filepath = '/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/vgg_16.hdf5'
model.load_weights(filepath)

In [ ]:
# early stoping
filepath = './output/mnist_cnn_best1.hdf5'
filecheck = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
              save_best_only=True, mode='max')

LRreduce = ReduceLROnPlateau(monitor='val_acc', patience=4, verbose=1, 
              factor=0.5, min_lr=0.00001)# acc

In [ ]:
Process = model.fit_generator(train_generator,
                steps_per_epoch= 45000 // batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_valid, y_valid),
                callbacks=[LRreduce,filecheck])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Submission functions
from keras.models import load_model
test_images = test_images.astype('float32').reshape(-1,row,col,1)
def get_submission(filename):
  # model = load_model('./output/mnist_cnn_best1.hdf5')
  model = load_model('/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/vgg_16_best.hdf5')
  
  ypred_onehot = model.predict(test_images)
  ypred = np.argmax(ypred_onehot,axis=1)

  with open(filename, 'w') as f:
    print('Id,Label', file=f)
    for i in range(len(ypred)):
      print(f'{i},{ypred[i]}', file=f)

def get_submission_voting(filename, weights, mode='hard'):

  model_1 = load_model('/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/CNN_8.hdf5')
  model_2 = load_model('/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/resnet_18.hdf5')
  model_3 = load_model('/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/vgg_16_best.hdf5')
  
  rst_1 = model_1.predict(x_test)
  rst_2 = model_2.predict(rgb_x_test)
  rst_3 = model_3.predict(x_test)

  rst_1_hard = keras.utils.to_categorical(np.argmax(model_1.predict(x_test), axis=1), num_classes)
  rst_2_hard = keras.utils.to_categorical(np.argmax(model_2.predict(rgb_x_test), axis=1), num_classes)
  rst_3_hard = keras.utils.to_categorical(np.argmax(model_3.predict(x_test), axis=1), num_classes)

  if mode == 'hard':
    pred_y = np.argmax(rst_1_hard*weights[0]+rst_2_hard*weights[1]+rst_3_hard*weights[2], axis=1)
  elif mode == 'soft':
    pred_y = np.argmax(rst_1*weights[0]+rst_2*weights[1]+rst_3*weights[2], axis=1)
  else:
    print('mode should be either \'hard\' or \'soft\'')
    return None

  with open(filename, 'w') as f:
    print('Id,Label', file=f)
    for i in range(len(pred_y)):

      print(f'{i},{pred_y[i]}', file=f)

In [ ]:
get_submission('output.csv')

In [ ]:
get_submission_voting('output.csv', weights=[1,1,1.5], mode='hard')

In [ ]:
model = load_model('./output/mnist_cnn_best1.hdf5')
  
ypred_onehot = model.predict(test_images)
ypred = np.argmax(ypred_onehot,axis=1)#

In [ ]:
# Ensembling - Voting

model_1 = load_model('/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/CNN_8.hdf5')
model_2 = load_model('/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/resnet_18.hdf5')
model_3 = load_model('/content/drive/My Drive/2019FALL_COMP551_MP3_GROUP37/Ensemble/vgg_16_best.hdf5')

In [ ]:
# get validation score for voting
from sklearn.metrics import accuracy_score

model = model_1

weights = [1,1,10]

rst_1 = model_1.predict(x_valid)
rst_2 = model_2.predict(rgb_x_valid)
rst_3 = model_3.predict(x_valid)

rst_1_hard = keras.utils.to_categorical(np.argmax(model_1.predict(x_valid), axis=1), num_classes)
rst_2_hard = keras.utils.to_categorical(np.argmax(model_2.predict(rgb_x_valid), axis=1), num_classes)
rst_3_hard = keras.utils.to_categorical(np.argmax(model_3.predict(x_valid), axis=1), num_classes)

pred_y = np.argmax(rst_1*weights[0]+rst_2*weights[1]+rst_3*weights[2], axis=1)
# pred_y = np.argmax(rst_1_hard*weights[0]+rst_2_hard*weights[1]+rst_3_hard*weights[2], axis=1)
print('Validation accuracy =', accuracy_score(pred_y, np.array(valid_y)))


In [ ]:
from sklearn.metrics import classification_report
from sklearn import model_selection, feature_extraction, preprocessing, svm, pipeline, metrics, tree, linear_model
print(metrics.classification_report(ytrue, ypred, digits=5))
print(ypred)

In [ ]:
!nvidia-smi
!ps -aux|grep python


In [ ]:
!kill -9 147